# Imports

First we import the discrete GRN dependencies.

In [4]:
from discrete_motif import DiscreteGrnMotif
import discrete_motif_functions as functions
import discrete_motif_measures as measures
import discrete_motif_generator as generator
import discrete_motif_plotting as visualize
from IPython.display import HTML, display
import random
import tabulate

# t-sne imports
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE

# significance tests import
import scipy.stats

# Data imports and sample

So far, we don't import data. In theory we want to compare three samples:

1. Random transition tables of the system state
2. Biological transition tables of the system state (still random samples, but with constraints)
3. Real GRN motifs

The last is difficult to draw a large sample from. Here we would import this sample, but I am focussing on the first two first.

In [2]:
def loop_impacts(network_size, nudge_size, motif):
    impacts = []
    synergy = None
    
    # try to find the synergy
    try:
        # find the synergy
        motif.evaluate_motif()
        mutual_information = measures.mutual_information(motif)
        synergy = synergy_measure(motif)

        # normalize by dividing by the total mutual information
        if float(mutual_information) != 0:
            synergy = float(synergy)/float(mutual_information)
        else:
            raise ValueError("divide by zero avoided")
    except ValueError:
        print("failed to find the synergy")
        synergy = None
    
    # try to loop
    for nudge_width in range(1, network_size + 1):
        try:
            # sample a set to nudge
            options = list(range(0, network_size))
            random.shuffle(options)
            targets = []
            for i in range(0, nudge_width):
                targets.append(options[i])
            targets.sort()
            
            # find the nudge impact
            motif.reset_to_state(0)
            motif.evaluate_motif()
            motif.reset_to_state(0)
            motif.nudge(targets, [], nudge_size)
            motif.evaluate_motif()
            
            # we compare the two evolved states
            impact = measures.abs_diff(motif.states[-3], motif.states[-1])
            impact_tuple = (nudge_width, impact)
            impacts.append(impact_tuple)
        except ValueError:
            print("failed to find the impact")
            impacts.append((nudge_width, None)) 
    
    return synergy, impacts

In [3]:
# in this cell we generate samples

samples = []
sample_size = 40
network_sizes = [2, 3, 4, 5]
logic_sizes = [2, 3, 4]
nudge_sizes = [0.2]
samples_rand = []

# set the synergy measure
synergy_measure = measures.synergy_middleground

# TODO: check my biological generation setting defaults
# TODO: remove or check the synergy-impact scatter for consistency

# draw a few completely random samples, with different parameters
for network_size in network_sizes:
    for logic_size in logic_sizes:
        for nudge_size in nudge_sizes:
            # let's store it in a dict, with some meta information
            print("Samping with %s nodes and %s-valued logic" % (network_size, logic_size))
            sample = {}
            sample["network_size"] = network_size
            sample["logic_size"] = logic_size
            sample["random_tables"] = generator.generate_random(sample_size, network_size, logic_size)
            sample["bio_tables"] = generator.generate_motifs(sample_size, network_size, logic_size)[0]
            
            # set transition tables
            # enrich the bio tables with synergy and a impact vector from 1 to system size
            
            # TODO: need to use targets
            # TODO: also turn this into a method
            
            counter = 0
            for motif in sample["bio_tables"]:
                motif.set_transition_table()
                
                synergy, impacts = loop_impacts(network_size, nudge_size, motif)
                
                sample["bio_tables"][counter] = (motif, synergy, impacts)
                counter += 1
                
            # enrich the random tables
            counter = 0
            for motif in sample["random_tables"]:
                synergy, impacts = loop_impacts(network_size, nudge_size, motif)
                
                sample["random_tables"][counter] = (motif, synergy, impacts)
                counter += 1

        # add to our set of samples
        samples.append(sample)

Samping with 2 nodes and 2-valued logic
failed to find the synergy
Samping with 2 nodes and 3-valued logic
failed to find the synergy
Samping with 2 nodes and 4-valued logic
Samping with 3 nodes and 2-valued logic
Samping with 3 nodes and 3-valued logic
Samping with 3 nodes and 4-valued logic
Samping with 4 nodes and 2-valued logic
could not find satisfying minus state randomly
[ 1.  1.]
0
[ 0.08640938  0.09460188]


IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
# we do not import real data yet

# Data visualisation

We use t-SNE to visualize our sample. What we see makes perfect sense: the biological motifs seem to be a subset of the sample space. In higher valued logic systems, the space becomes larger, making the sample size insufficient. As t-SNE also does clustering, it then starts to appear that we have two seperate sample, but this is simply the separation from the very similar biological motifs from the rest.

In [ ]:
# x_train should be an array of N vectors of length M
# where N is the sample size and M is the length of the combined new state of the transition table
# we combine both the random and bio samples, but give them different colors

for i in range(0, len(samples)):
    x_train = []
    y_color = []
    labels = []
    # the random tables
    for sample in samples[i]["random_tables"]:
        table = sample[0]
        sample_vector = []
        for row in table.transition_table:
            sample_vector.extend(row[(len(row)/2):])
        y_color.append("red")
        x_train.append(sample_vector)
        labels.append("Random transition table")

    # the bio table
    for sample in samples[i]["bio_tables"]:
        motif = sample[0]
        sample_vector = []
        for row in motif.transition_table:
            sample_vector.extend(row[(len(row)/2):])
        y_color.append("blue")
        x_train.append(sample_vector)
        labels.append("Biological transition table")

    x_train = np.array(x_train)
    title = "Two-dimensional embedding of transition tables with motif size %s and %s-valued logic" % (samples[i]["network_size"], samples[i]["logic_size"])
    x_train_embedded = TSNE(n_components=2, perplexity=40, early_exaggeration=1, verbose=0).fit_transform(x_train)
    visualize.plot_scatter(x_train_embedded, y_color, labels, title)

# Experiments

## Correlation between synergy and single nudge resilience in a truly random motif

We use the WMS synergy, which is an upper bound.

In [ ]:
for i in range(0, len(samples)):
    # a;ready dod
    
    

## Correlation between synergy and wide nudge resilience in a truly random motif

We also do a wide nudge.

In [ ]:
# let's check the first one out
motif_rand = motifs[0]
motif_rand.evaluate_motif()
print("The states: ")
for state in motif_rand.states:
    print(state)
print("The correlations: ")
print(motif_rand.grn_vars["correlations"])
print("Transitions: ")
print(motif_rand.transition_table)

# make a plot with WMS synergy
visualize.scatterplot_synergy_nudgeimpact(motifs, 3, 0.5, measures.synergy_middleground)

# Experiments

## Correlation between synergy and nudge resilience in biological random motif

We use the middle between the  WMS synergy, which is a lower bound, and the maximum individiual MI, which is an upper bound.

In [ ]:
# create a network motif
motifs, indegree_avg = generator.generate_motifs(samplesize=500, no_nodes=3, numvalues=3, conflict_rule='totaleffect')
print("average indegree is "+str(indegree_avg))

# make a plot with WMS synergy
visualize.scatterplot_synergy_nudgeimpact(motifs, 3, 0.5, measures.synergy_middleground)

In [ ]:
# create a network motif
motifs, indegree_avg = generator.generate_motifs(samplesize=500, no_nodes=3, numvalues=3, conflict_rule='totaleffect')
print("average indegree is "+str(indegree_avg))

# make a plot with WMS synergy
visualize.scatterplot_synergy_nudgeimpact(motifs, 3, 0.5, measures.synergy_middleground)

In [ ]:
# create a network motif
#motifs, indegree_avg = generator.generate_motifs(samplesize=1, no_nodes=3, numvalues=5, conflict_rule='totaleffect')
#print("average indegree is "+str(indegree_avg))

# make a plot with WMS synergy
#visualize.scatterplot_synergy_nudgeimpact(motifs, 3, 0.5, measures.synergy_quax)

## More synergy in biological motif than in random

In [ ]:
#scipy.stats.ttest_ind

## Real GRN motif scores better than a random motif in memory

In [ ]:
# define memory measure

# scipy.stats.ttest_ind

## Real GRN motif scores better than a random motif in single nudge resilience

In [ ]:
#scipy.stats.ttest_ind

## Real GRN motif scores similar to random motif in multiple nudge resilience

In [ ]:
#scipy.stats.ttest_ind

## There is a stronger than linear decrease in resilience when increasing the number of variables nudged in a biological GRN motif

In [ ]:
# make sure we do the nudge impact for every

# make lineplot with error bars